In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil
import sys

import flopy
from modflowapi import ModflowApi

from bmi.wrapper import BMIWrapper

In [ ]:
sys.path.append("../common")
from liss_settings import libmf6, get_dflow_grid_name, get_dflow_dtuser, get_modflow_coupling_tag

In [ ]:
control_path = pl.Path("../dflow-fm/coarse/tides/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
grid_name = get_dflow_grid_name(control_path)
print(grid_name)

In [ ]:
dflowfm_dtuser = get_dflow_dtuser(control_path)
print(dflowfm_dtuser)

#### Set unit conversion factors

In [ ]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((3.28082**3) * 86400.)

#### Set the MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [ ]:
mf_couple_freq_hours = 1.0  #Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
dflow_per_mf = int(mf_couple_freq / dflowfm_dtuser)

In [ ]:
print(f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\nMODFLOW coupled to D-FLOW FM every {dflow_per_mf} output time step ({dflowfm_dtuser} sec.)") 

In [ ]:
mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

In [ ]:
nstp = int(86400.0 / (dflow_per_mf * dflowfm_dtuser))
print(f"MODFLOW time steps per day: {nstp}")

#### Set a few variables for controlling coupling

In [ ]:
HDRY = -1e30
DEPTH_MIN = 0.1

#### Print the path of the modflow6 shared library

In [ ]:
str(libmf6), libmf6.is_file()

#### Load the D-FLOW to MODFLOW weights

In [ ]:
npzfile = np.load(f"../mapping/dflow2mfghb_{grid_name}.npz")
npzfile

In [ ]:
dflow2mfghb = npzfile["dflow2mfghb"]
ghbmask = npzfile["ghbmask"]
ghb2qext = npzfile["ghb2qext"]

#### Define paths for the model simulation

In [ ]:
mf_base_path = pl.Path("../modflow/greenport500ft/base/").resolve()
mf_run_path = pl.Path(f"../modflow/greenport500ft/run_{mf_tag}/").resolve()

#### Load the base MODFLOW model

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_base_path)
gwf = sim.get_model()

In [ ]:
sim.set_sim_path(mf_run_path)

#### Change TDIS to defined time steps

In [ ]:
tdis = sim.get_package("TDIS")
perioddata = tdis.perioddata.array

In [ ]:
perioddata["nstp"] = nstp
tdis.perioddata = perioddata

#### Write the new model files

In [ ]:
sim.write_simulation()

#### Define base GHB variables

In [ ]:
ghb_data0 = gwf.ghb.stress_period_data.get_dataframe()[0]
ghb_data0

#### Setup and initialize D-FLOW FM

You will need to set `dflow_dirpath` to the correct directory on your machine.

In [ ]:
# dflow_root_path = pl.Path(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM").resolve()
# dll_paths = (
#     r"plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin",
#     r"plugins\DeltaShell.Dimr\kernels\x64\share\bin",
# )

In [ ]:
# # os.environ["PATH"] = (
# #     str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
# # )
# old_path = os.environ["PATH"]
# os.environ["PATH"] = ""
# path = ""
# for p in dll_paths:
#     path += f"{dflow_root_path / p}" + os.pathsep
# os.environ["PATH"] = path + old_path

In [ ]:
os.environ["PATH"]

In [ ]:
# dflow_dirpath = dflow_root_path / f"{dll_paths[0]}"

In [ ]:
# dflow_dirpath = os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin")
# dflow_deps_dirpath = (
#     os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\share\bin"),
# )
dflow_dirpath = pl.Path(r"X:\Work\compound_flooding\dflow-fm\dflowfm_dll") 
dflow_base = pl.Path(r"X:\Work\compound_flooding\dflow-fm\coarse\tides\base").resolve()
dflow_working = pl.Path(r"X:\Work\compound_flooding\dflow-fm\coarse\tides\run").resolve()
dflow_config = dflow_working / "FlowFM.mdu"

In [ ]:
if dflow_working.is_dir():
    shutil.rmtree(dflow_working)
shutil.copytree(dflow_base, dflow_working)
(dflow_working / "output").mkdir(parents=True, exist_ok=True)

In [ ]:
os.environ["PATH"]

In [ ]:
# Add dflowfm dll folder to PATH so that it can be found by the BMIWrapper
os.environ["PATH"] = (
    str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
)

In [ ]:
# os.environ["PATH"]

In [ ]:
(pl.Path(dflow_dirpath) / "dflowfm.dll").is_file()

#### Initialize D-Flow FM

In [ ]:
dflowfm = BMIWrapper(
    engine="dflowfm",
    configfile=str(dflow_config),
)

In [ ]:
dflowfm.initialize()

#### Get data from D-FLOW FM

In [ ]:
ndxi = int(dflowfm.get_var("ndxi"))
ndx = int(dflowfm.get_var("ndx"))
x = dflowfm.get_var("xz")
y = dflowfm.get_var("yz")
z = dflowfm.get_var("bl")
xy = [(xx, yy) for (xx, yy) in zip(x, y)]
ndx, ndxi, x.shape, y.shape

In [ ]:
dflowfm.get_var("s1")

In [ ]:
v = dflowfm.get_var("hs")
v.shape, v

In [ ]:
qext = np.zeros(ndx)
qext.shape, qext

In [ ]:
# dflowfm.set_var("qext", qext)

In [ ]:
qext_cum = np.zeros(ndx)
qext_cum.shape

In [ ]:
vextcum = dflowfm.get_var("vextcum")
vextcum.shape, vextcum

#### Initialize MODFLOW using MODFLOW API

In [ ]:
mf6 = ModflowApi(str(libmf6), working_directory=mf_run_path)

In [ ]:
mf6.initialize()

#### Define MODFLOW variable tags and set pointer to MODFLOW variables

In [ ]:
ghb_bhead_tag = mf6.get_var_address("BHEAD", "MODFLOW", "GHB-1")
ghb_cond_tag = mf6.get_var_address("COND", "MODFLOW", "GHB-1")
ghb_flow_tag = mf6.get_var_address("SIMVALS", "MODFLOW", "GHB-1")

In [ ]:
ghb_bhead_ptr = mf6.get_value_ptr(ghb_bhead_tag)
ghb_cond_ptr = mf6.get_value_ptr(ghb_cond_tag)
ghb_flow = np.zeros(ghb_bhead_ptr.shape)

#### Create dictionaries for saving modified GHB data

In [ ]:
elev_dict = {}
cond_dict = {}
qext_dict = {}

#### Function to update MODFLOW GHB data

In [ ]:
def update_mf(key, s, d):
    mask = d == 0.0
    s[mask] = 0.0
    mult = np.full(d.shape, 1.0)
    mult[mask] = 0.0

    ghb_head = ghb_data0["bhead"].to_numpy()
    ghb_head[ghbmask] = dflow2mfghb.dot(s)[ghbmask] * m2ft
    ghb_cond = ghb_data0["cond"].to_numpy()
    ghb_cond[ghbmask] = ghb_cond[ghbmask] * dflow2mfghb.dot(mult)[ghbmask]
    
    ghb_bhead_ptr[:] = ghb_head[:]
    ghb_cond_ptr[:] = ghb_cond[:]

    # update results dictionary
    elev_dict[key] = ghb_head.copy()
    cond_dict[key] = ghb_cond.copy()

#### Function to update D-Flow FM Qext data

In [ ]:
def update_dflow(key, d):
    ghb_flow = -mf6.get_value(ghb_flow_tag) * cfd2cms
    dflow_qext = ghb2qext.dot(ghb_flow)
    dflow_qext[d == 0.0] = 0.0
    qext_cum[:ndxi] += dflow_qext[:ndxi]

    qext[:ndxi] = dflow_qext[:ndxi]
    dflowfm.set_var("qext", qext)

    # update results dictionaries
    qext_dict[key] = qext[:ndxi].copy()


#### Run each time step

In [ ]:
print(
    f"DFLOWFM current_time: {dflowfm.get_current_time():15,.1f} sec. ({dflowfm.get_current_time()/86400.:15,.1f} days)\n"
     + f"DFLOWFM end_time:     {dflowfm.get_end_time():15,.1f} sec. ({dflowfm.get_end_time()/86400.:15,.1f} days)"
)

In [ ]:
idx = 0
jdx = 0
t0 = time.perf_counter()
current_time = dflowfm.get_current_time()
end_time = dflowfm.get_end_time()
while current_time <= end_time:
    idx += 1
    ontime = dflowfm.get_current_time()
    onday = ontime / 86400.
    frac_comp = current_time / end_time
    print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
    dflowfm.update()

    current_time = dflowfm.get_current_time()
    if idx == int(dflow_per_mf):
        print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d}) ***", end="\r")
        s = dflowfm.get_var("s1")[:ndxi]
        d = dflowfm.get_var("hs")[:ndxi]

        mf6.prepare_time_step(mf6.get_time_step())
        update_mf(str(jdx), s, d)
        mf6.do_time_step()
        mf6.finalize_time_step()
        update_dflow(str(jdx), d)

        idx = 0
        jdx += 1

    
    if current_time == end_time:
        break

vextcum = dflowfm.get_var("vextcum")

t1 = time.perf_counter()
print(f"\nrun time: {(t1 - t0) / 60.} min")

In [ ]:
# Finalize
mf6.finalize()

In [ ]:
dflowfm.finalize()

#### Save ghb elevation and conductance data to compressed files

In [ ]:
np.savez_compressed(f"{mf_run_path}/ghb_elev.npz", **elev_dict)
np.savez_compressed(f"{mf_run_path}/ghb_cond.npz", **cond_dict)
np.savez_compressed(f"{mf_run_path}/qext.npz", **qext_dict)